# Applying Euclidean distance only on chord progressions / with centralization, scaling, boxcox

In [85]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [86]:
transitionMatrices = pd.read_csv('./data/transitionMatricesNotNormalized.csv')

In [87]:
df

,Unnamed: 0,id,0,1,2,4,5,6,7,8,...,1276,1278,1281,1286,1287,1288,1290,1292,1294,1295
0,0,202646.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1234782.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,877151.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1425968.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,808812.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,3477,79685.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3478,3478,39828.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3479,3479,957714.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3480,3480,1172824.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
transitionMatrices.sum(axis = 1).sum(axis=0)

3481.999999999998

In [64]:
transitionMatrices['id'] = transitionMatrices.index

In [65]:
transitionMatrices[transitionMatrices.id == 1686718]

,0,1,2,4,5,6,7,8,9,11,...,1278,1281,1286,1287,1288,1290,1292,1294,1295,id
id,,,,,,,,,,,,,,,,,,,,,
1686718,0.035714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1686718


In [66]:
transitionMatrices.reset_index(drop=True, inplace=True)

In [67]:
ids = transitionMatrices.id
df = transitionMatrices.iloc[:, 0:-1]

## Removing Skewness with boxcox

In [68]:
df = df + 1
cols = df.columns.tolist()

In [69]:
for colName in cols:
    df[colName] = stats.boxcox(df[colName])[0]

In [70]:
df

,0,1,2,4,5,6,7,8,9,11,...,1276,1278,1281,1286,1287,1288,1290,1292,1294,1295
0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.043438,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.003224,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.021288,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.011857,0.0,0.0,0.0,0.0,0.000000,0.000931,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3478,0.028829,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3479,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3480,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Combining Centering and Scaling

In [71]:
cols = df.columns
scaler = StandardScaler()
scaler.fit(df)
data_normalized = scaler.transform(df)
print('mean: ', data_normalized.mean(axis=0).round(2))
print('std: ', data_normalized.std(axis=0).round(2))

df = pd.DataFrame(data_normalized, columns=cols)

mean:  [ 0. -0. -0. -0. -0. -0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0. -0.  0.
  0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  0. -0. -0.  0. -0.  0.  0.
  0. -0. -0.  0. -0. -0. -0.  0. -0. -0. -0.  0.  0. -0. -0. -0. -0. -0.
 -0.  0.  0. -0. -0. -0.  0.  0.  0. -0.  0. -0.  0.  0. -0. -0.  0.  0.
  0.  0. -0. -0. -0. -0. -0.  0.  0. -0.  0. -0. -0. -0.  0.  0. -0. -0.
  0. -0. -0.  0.  0. -0. -0.  0.  0. -0. -0. -0.  0. -0.  0.  0.  0.  0.
  0. -0. -0.  0.  0. -0.  0.  0.  0. -0.  0. -0.  0.  0.  0. -0.  0.  0.
  0. -0.  0. -0. -0.  0.  0. -0. -0. -0. -0. -0.  0.  0.  0. -0. -0.  0.
 -0.  0.  0.  0. -0.  0.  0.  0. -0.  0. -0. -0. -0. -0.  0. -0. -0. -0.
  0.  0. -0.  0. -0.  0. -0.  0.  0. -0. -0.  0.  0.  0.  0. -0. -0. -0.
  0.  0.  0. -0. -0. -0. -0. -0. -0.  0.  0. -0. -0.  0.  0.  0. -0.  0.
 -0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0. -0. -0.  0. -0. -0. -0.
 -0. -0.  0. -0.  0.  0. -0.  0.  0. -0.  0.  0. -0.  0.  0.  0. -0. -0.
  0. -0. -0.  0.  0. -0. -0.  0. -0. -0. -0.

In [72]:
df

,0,1,2,4,5,6,7,8,9,11,...,1276,1278,1281,1286,1287,1288,1290,1292,1294,1295
0,-0.758364,-0.096309,-0.047988,-0.023973,-0.09777,-0.271655,-0.143233,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547
1,1.934643,-0.096309,-0.047988,-0.023973,-0.09777,-0.271655,-0.143233,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547
2,-0.758364,-0.096309,-0.047988,-0.023973,-0.09777,3.749167,-0.143233,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547
3,0.561450,-0.096309,-0.047988,-0.023973,-0.09777,-0.271655,-0.143233,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547
4,-0.023289,-0.096309,-0.047988,-0.023973,-0.09777,-0.271655,6.981854,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,-0.758364,-0.096309,-0.047988,-0.023973,-0.09777,-0.271655,-0.143233,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547
3478,1.028961,-0.096309,-0.047988,-0.023973,-0.09777,-0.271655,-0.143233,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547
3479,-0.758364,-0.096309,-0.047988,-0.023973,-0.09777,-0.271655,-0.143233,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547
3480,-0.758364,-0.096309,-0.047988,-0.023973,-0.09777,-0.271655,-0.143233,-0.033913,-0.029365,-0.493782,...,-0.029365,-0.041547,-0.109156,-0.016949,-0.083309,-0.016949,-0.016949,-0.076007,-0.016949,-0.041547


## Applying eucledian distance

In [73]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [74]:
originalSongId = 1686718
originalSongIndex = ids[ids == originalSongId].index[0]
originalSongIndex

2847

***Dropping the cover song***

In [75]:
coverSongId = 180849 # THe Scientist by Boyce Avenue
coverIndex = ids[ids == coverSongId].index[0]
coverSongVector = df.iloc[coverIndex, :]

In [76]:
ranking = []
for index in range(0, len(df)):
    ranking.append([ids[index], euclidean_distance(df.iloc[index, :], coverSongVector)])
ranking    

[[23, 38.92750267747127],
 [105, 28.054119775540293],
 [118, 33.69914160903405],
 [120, 31.12188414855512],
 [292, 50.75060874756118],
 [407, 30.107949001271386],
 [418, 35.16624191683042],
 [456, 28.134622154049737],
 [467, 53.22587757036546],
 [469, 37.97639744731765],
 [588, 26.67253390714992],
 [628, 42.018036716898095],
 [635, 27.319669767380528],
 [652, 33.30411512799624],
 [816, 28.730044629648045],
 [830, 54.238521664359794],
 [836, 29.017301908685727],
 [1334, 28.097267130905397],
 [1345, 27.48033936824999],
 [1395, 27.619784464132415],
 [1433, 28.390818192049718],
 [2063, 30.07956298235855],
 [2147, 31.03021058305956],
 [2148, 39.21824510948047],
 [2220, 28.72824776192003],
 [2535, 26.858765509143964],
 [2553, 26.959465984447892],
 [3118, 30.21060155009148],
 [3165, 27.167238847409116],
 [3337, 27.44982187181128],
 [3391, 27.530934722253562],
 [3991, 26.871528241128235],
 [4019, 27.399937382195397],
 [4041, 27.550905633067785],
 [4084, 28.06292498416371],
 [4169, 32.825561585

In [77]:
ranksDF = pd.DataFrame(ranking)

In [78]:
ranksDF = ranksDF.rename(columns={0: "id", 1: "rank"})

In [79]:
ranksDF

,id,rank
0,23,38.927503
1,105,28.054120
2,118,33.699142
3,120,31.121884
4,292,50.750609
...,...,...
3477,3781055,27.735988
3478,3860363,29.879806
3479,3898172,30.786732
3480,3911342,27.838075


In [80]:
ranksDF = ranksDF.sort_values(by="rank")

In [81]:
ranksDF.reset_index(drop=True, inplace=True)

In [82]:
ranksDF

,id,rank
0,180849,0.000000
1,1119299,21.794960
2,610562,24.315368
3,1127345,25.993370
4,748267,26.005278
...,...,...
3477,128462,164.994722
3478,1716013,174.848868
3479,79223,182.847402
3480,106028,191.909329


In [83]:
ranksDF[ranksDF['id']==originalSongId]

,id,rank
450,1686718,27.10298


## Rank for original Song 450